In [11]:
## Get the data from api call as json
import requests 
r = requests.get('https://api.coinmarketcap.com/v1/ticker/?limit=0')
json_record= r.json()
print json_record

[{u'market_cap_usd': u'140597002301', u'price_usd': u'8280.44', u'last_updated': u'1523816672', u'name': u'Bitcoin', u'24h_volume_usd': u'5214680000.0', u'percent_change_7d': u'17.7', u'symbol': u'BTC', u'max_supply': u'21000000.0', u'rank': u'1', u'percent_change_1h': u'0.23', u'total_supply': u'16979412.0', u'price_btc': u'1.0', u'available_supply': u'16979412.0', u'percent_change_24h': u'3.85', u'id': u'bitcoin'}, {u'market_cap_usd': u'51980306667.0', u'price_usd': u'525.909', u'last_updated': u'1523816658', u'name': u'Ethereum', u'24h_volume_usd': u'1663210000.0', u'percent_change_7d': u'31.64', u'symbol': u'ETH', u'max_supply': None, u'rank': u'2', u'percent_change_1h': u'0.54', u'total_supply': u'98838975.0', u'price_btc': u'0.063379', u'available_supply': u'98838975.0', u'percent_change_24h': u'5.87', u'id': u'ethereum'}, {u'market_cap_usd': u'26244705426.0', u'price_usd': u'0.670829', u'last_updated': u'1523816641', u'name': u'Ripple', u'24h_volume_usd': u'612925000.0', u'perce

In [7]:
### Convert json to csv for loading into bigquery
import csv 
outfile = open("crypto.csv",'w+')
writer = csv.writer(outfile)

schema = ['market_cap_usd', 'price_usd', 'last_updated', 'name', '24h_volume_usd', 'percent_change_7d', 'symbol', 'max_supply', 'rank', 'percent_change_1h', 'total_supply', 'price_btc', 'available_supply', 'percent_change_24h', 'id']


writer.writerow(schema)
for record in json_record:

#	print record 
  out_row = []

  for key in schema:
      #print record[key]


      out_row.append(record[key])

  writer.writerow(out_row)
  




In [43]:
import google.datalab.bigquery as bq
## I loaded the dataset using the UI, I can load it using the bq module but out of the sake of time I decided to skip doing this.

market_cap_usd,price_usd,last_updated,name,_24h_volume_usd,percent_change_7d,symbol,max_supply,rank,percent_change_1h,total_supply,price_btc,available_supply,percent_change_24h,id
104464.0,0.0103797,1523809745,VirtualCoin,,-27.95,VC,4000000000.0,1163,,10064287.0,1.25e-06,10064287.0,4.05,virtualcoin
740392.0,0.148842,1523793564,EtherSportz,,15.26,ESZ,,1020,,10000000.0,1.784e-05,4974347.0,8.25,ethersportz
601523.0,0.0707375,1523754559,EthBet,,26.59,EBET,,1030,,10000000.0,8.79e-06,8503600.0,1.61,ethbet
126283.0,0.0140314,1523343257,Confido,,-25.93,CFD,15000000.0,1148,,15000000.0,2.09e-06,9000000.0,,confido
57641.0,0.0477436,1523676843,Uro,,17.04,URO,,1206,,1207310.0,5.95e-06,1207310.0,,uro
984382.0,0.01693,1523513655,FuckToken,,,FUCK,,1008,,70856019.0,2.44e-06,58144264.0,,fucktoken
410879.0,1.94909,1523734746,Flycoin,,14.66,FLY,,1053,,210806.0,0.00024346,210806.0,0.3,flycoin
5179563.0,0.336033,1522169644,Vcash,,,XVC,,965,,15413851.0,4.153e-05,15413851.0,,vcash
925894.0,0.0476712,1523741956,Nexxus,,198.68,NXX,,1010,,318000000.0,5.92e-06,19422507.0,2.03,nexxus
62537.0,0.000749392,1523796547,VIP Tokens,,51.82,VIP,,1200,,83450403.0,9e-08,83450403.0,4.33,vip-tokens


In [19]:
%%bq query
-- The count of crypto with price in USD > 8k 
-- Its in the gutters :( 
SELECT  count(*) FROM Crypto.crypto_currencies  where price_usd > 8000



f0_
0


In [18]:
%%bq query
-- Total market cap of top 100 crypto in market cap 
select sum(market_cap_usd) from (
select * from Crypto.crypto_currencies order by market_cap_usd desc limit 100) a;

f0_
3.01048805156e+11


In [13]:
%%bq query
-- All cryptos with available supper < 5 mil
select name,available_supply from Crypto.crypto_currencies where available_supply < 5000000


name,available_supply
DigixDAO,2000000.0
Zcash,3700231.0
Byteball Bytes,645222.0
Maker,618228.0
Veritaseum,2036645.0
Mixin,411682.0
Elastos,4975104.0
ZCoin,4528741.0


In [15]:
%%bq query
-- Top 5 currencies with higest percent change is last week
select name,percent_change_7d from Crypto.crypto_currencies order by percent_change_7d desc limit 5

name,percent_change_7d
Golem,116.62
Mithril,107.45
Bytom,63.68
Wanchain,59.33
DigiByte,59.15


In [17]:
%%bq query
-- Crypto with X in symbol
select count(symbol) from Crypto.crypto_currencies where symbol like '%X%'

f0_
15
